In [ ]:
import os
import re
import numpy as np
from rdkit import Chem
from tqdm import tqdm
import pandas as pd
from rdkit.Chem import Draw
import sys
from spectral_denoising.file_io import *
from spectral_denoising.spectral_denoising import *
from spectral_denoising.spectra_plotter import *
from spectral_denoising.chem_utils import *
from spectral_denoising.constant import proton_mass
from spectral_denoising.test import *
from rdkit.Chem.Draw import IPythonConsole
sys.path.append(os.path.join(".."))
%load_ext autoreload
%autoreload 2

In [ ]:
nist_test = read_df('/Users/fanzhoukong/Documents/GitHub/Libgen_data/curated_library/csv/nist_pos_sample.csv')

In [6]:
instance = nist_test.iloc[1]

In [ ]:
instance

In [ ]:
msms = instance['spectrum']
ms2_plot(msms)

In [307]:

    # sum_values = [int(value) for key, value in formula_dict.items() if key != 'H']
    # return (sum_values)

In [ ]:
get_rarest_heavy_atom('C6H1O2')

In [ ]:
count

In [235]:
formula_array = np.array(list(dict.items()), dtype=[('key', 'U2'), ('value', 'int')])

In [ ]:
np.array()

In [ ]:
[(dict.keys()), dict.values()]

In [ ]:
np.argmin(np.array(dict.values()))

In [21]:
all_possible_subformulas, all_possible_masses = get_all_subformulas(instance['SMILES'])

In [170]:
idx_left, idx_right = all_possible_masses.searchsorted([neutral_mass-0.005, neutral_mass+0.005])

In [ ]:
target_formula = all_possible_subformulas[idx_left:idx_right][0]
target_formula

In [172]:
subgraph_length = get_subgraph_length(all_possible_subformulas[idx_left:idx_right][0])

In [32]:
mol = Chem.MolFromSmiles(instance['SMILES'])

In [195]:
lst = Chem.rdmolops.FindAllSubgraphsOfLengthN(submol, 10)

In [ ]:
len(lst)

In [173]:
all_subgraphs_given_length = Chem.rdmolops.FindAllSubgraphsOfLengthN(mol, subgraph_length)

In [ ]:
mol

In [187]:
submol=Chem.PathToSubmol(mol, env)

In [ ]:
all_subgraphs_given_length(submol, 10)

In [ ]:
Draw.MolsToGridImage([mol], subImgSize=(500,500),
                     highlightBondLists=[[a for a in env]], 
                     molsPerRow=1)

In [ ]:
CalcMolFormula(mol)

In [174]:
lst = []
for subgraph in all_subgraphs_given_length:
    
    submol = Chem.PathToSubmol(mol, subgraph)

    sub_formula = CalcMolFormula(submol)
    if compare_formula(target_formula, sub_formula):
        lst.append(subgraph)

In [ ]:
mol = Chem.MolFromSmiles(instance['SMILES'])
IPythonConsole.molSize = (800, 800) 
Draw.MolsToGridImage([mol], subImgSize=(500,500),
                     highlightBondLists=[[a for a in lst[0]]], 
                     molsPerRow=1)


In [138]:
from spectral_denoising.seven_golden_rules import *

In [ ]:
check_ratio('C3O8H12')

In [149]:
fake_mass = Formula('C3O6H6').isotope.mass

In [150]:
idx_left, ldx_right = all_possible_masses.searchsorted([fake_mass-0.005, fake_mass+0.005])

In [ ]:
subformula_candidate = all_possible_subformulas[idx_left:ldx_right]
print(subformula_candidate)

In [158]:
import spectral_denoising.spectral_operations as so

In [160]:
msms_s = so.standardize_spectra(msms)

In [161]:
noise = so.pack_spectra([fake_mass+proton_mass], [0.5])

In [313]:
neutral_mass = msms_s.T[0][-1]-proton_mass


In [316]:
idx_left, idx_right = all_possible_masses.searchsorted([neutral_mass-0.005, neutral_mass+0.005])

In [335]:
from spectral_denoising.seven_golden_rules import *
from spectral_denoising.test import *

In [347]:
target_formula = all_possible_subformulas[idx_left:idx_right][0]
rarest_atom = get_rarest_heavy_atom(target_formula)
atom_idx_lst = get_atom_index(mol, rarest_atom)
subgraph_length = get_subgraph_length(target_formula)

In [391]:
env=Chem.FindAtomEnvironmentOfRadiusN(mol, subgraph_length, 1)

In [395]:
atoms=mol.GetAtoms()

In [ ]:
for a in atoms:
    print(a.GetSymbol(), a.GetIdx())
    break

In [407]:
mol_frags = rdmolops.GetMolFrags(mol, asMols = True)
# b
# largest_mol = max(mol_frags, default=index2smiles, key=lambda m: m.GetNumAtoms())
# largest_mol_smiles = Chem.MolToSmiles(largest_mol)

In [ ]:
subgraph_length

In [426]:
all_subgraph_env = []
for idx in tqdm(atom_idx_lst):
        subgraphs =Chem.rdmolops.FindAllSubgraphsOfLengthN(mol, subgraph_length, rootedAtAtom=idx)
        # break

In [427]:
subgraphs =Chem.rdmolops.FindAllSubgraphsOfLengthN(mol, subgraph_length)

In [425]:
subgraphs =Chem.rdmolops.FindAllSubgraphsOfLengthN(mol, subgraph_length, rootedAtAtom=atom_idx_lst[0])

In [ ]:
import networkx as nx
import numpy as np
from itertools import combinations

def generate_subgraphs_with_node(adjacency_matrix, target_node, length):
    # Convert adjacency matrix to NetworkX graph
    G = nx.from_numpy_matrix(np.array(adjacency_matrix))
    
    # Get all subgraphs of a specific length containing the target node
    subgraphs = []
    nodes = set(G.nodes())
    
    # Only consider combinations of nodes that include the target node
    nodes.remove(target_node)
    for node_comb in combinations(nodes, length - 1):
        subgraph_nodes = set(node_comb).union({target_node})
        subgraph = G.subgraph(subgraph_nodes)
        
        if len(subgraph) == length:
            subgraphs.append(subgraph)
    
    return subgraphs

# Example usage
adjacency_matrix = [
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 1, 1, 0]
]
def visualize_graph_from_adjacency(adjacency_matrix):
    # Create a NetworkX graph from adjacency matrix
    G = nx.from_numpy_matrix(np.array(adjacency_matrix))
    pos = nx.spring_layout(G)
    plt.figure(figsize=(6, 6))
    nx.draw_networkx(G, with_labels=True)
    plt.title('Graph from Adjacency Matrix')
    plt.show()
target_node = 1
length = 3

subgraphs = generate_subgraphs_with_node(adjacency_matrix, target_node, length)

# Print all subgraphs as edge lists
for i, sg in enumerate(subgraphs):
    print(f"Subgraph {i+1}: Nodes {list(sg.nodes())}, Edges {list(sg.edges())}")


In [442]:
adjacency_matrix = Chem.GetAdjacencyMatrix(mol)

In [450]:
G = nx.from_numpy_matrix((adjacency_matrix))

In [ ]:
nx.draw_networkx(G, with_labels=True)
    

In [ ]:
env

In [ ]:
len(sub_graphs)

In [ ]:
for f in all_possible_subformulas[idx_left:idx_right]:
    print(f, check_ratio(f), check_senior(f))

In [152]:
subgraph_length = get_subgraph_length(subformula_candidate[0])
all_subgraphs_given_length = Chem.rdmolops.FindAllSubgraphsOfLengthN(mol, subgraph_length)

In [153]:
lst = []
for subgraph in all_subgraphs_given_length:
    
    submol = Chem.PathToSubmol(mol, subgraph)

    sub_formula = CalcMolFormula(submol)
    if compare_formula(subformula_candidate[0], sub_formula):
        lst.append(subgraph)

In [ ]:
len(lst)